## Item 22: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

* Python's classes and inheritance make it easy to express your program's intended behaviors with objects.
* they allow you to improve and expand functionality over time.
* They provide flexibility in an environment of changing requirements.
* Knowing how to use them well enables you to write maintainable code.

* You want to record the grades of a set of students whose names aren't known in advance.
* You can define a class to store the names in a dictionary instread of using a predefined attribute for each student.

In [ ]:
from collections import defaultdict
from collections import namedtuple

from statistics import mean

### Dictionary

In [ ]:
grades = {
    'student_A': [10, 20, 30],
    'student_B': [20, 30, 40],
    'student_C': [30, 40, 50],
    'student_D': [40, 50, 60]
}

grades

In [ ]:
grades["student_A"].append(80)
grades["student_B"].append(30)
grades["student_C"].append(50)
grades["student_D"].append(60)

In [ ]:
grades

In [ ]:
avg_dict = {}

for student, score in grades.items():
    avg_dict[student] = sum(score)/ len(score)
    
avg_dict

### Statistics mean

In [ ]:
for student, score in grades.items():
    avg_dict[student] = mean(score)
    
avg_dict

### Simple class

In [ ]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

* Using the class is simple.

In [ ]:
book = SimpleGradebook()
book.add_student("student_E")
book.report_grade("student_E", 90)
book.report_grade("student_E", 95)
book.report_grade("student_E", 85)

In [ ]:
print(book.average_grade("student_E"))

### defaultdict

In [ ]:
g = [('a', 90), ('a', 85), ('a', 95), ('b', 90), ('b', 85)]

In [ ]:
d = {}

for k, v in g:
    d.setdefault(k, []).append(v)

In [ ]:
sorted(d.items())

In [ ]:
# better
d = defaultdict(list)

for k, v in g:
    d[k].append(v)

In [ ]:
sorted(d.items())

### By subject

* Extend the `SimpleGradebook` class to keep a list of grades by subject.
* By changing the _grades dictionary to map student names (the keys) to yet another dictionary (the values).
* The innermost dictionary will map subjects (the keys) to grades (the values).

In [ ]:
class BySubjectGradebook:
    def __init__(self):
        self._grades = {}  # Outer dict
        
    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # Inner dict
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        # print(by_subject.items())
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)   
        return total / count

* Using the class remains simple.

In [ ]:
book = BySubjectGradebook()
book.add_student("student_F")
book.report_grade("student_F", "Math", 80)
book.report_grade("student_F", "Math", 90)
book.report_grade("student_F", "Chemistry", 80)
book.report_grade("student_F", "Chemistry", 90)
book.report_grade("student_F", "Biology", 70)
book.report_grade("student_F", "Biology", 90)

In [ ]:
print(book.average_grade("student_F"))

### Add weight

In [ ]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = defaultdict(list)
        
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        # print(by_subject.items())
        
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            
            score_sum += subject_avg / total_weight
            score_count += 1
            
        return score_sum / score_count

* Using the class has also gotten more difficult.
* It's unclear what all of the members in the positional arguments mean.

In [ ]:
book = WeightedGradebook()
book.add_student("student_G")
book.report_grade("student_G", "Math", 70, 0.10)
book.report_grade("student_G", "Math", 80, 0.10)
book.report_grade("student_G", "Chemistry", 90, 0.10)
book.report_grade("student_G", "Chemistry", 95, 0.10)

In [ ]:
print(book.average_grade("student_G"))

* Using the class has also gotten more difficult.
* It's unclear what all of the numbers in the positional arguments mean.
* When you see complexity like this happen, it's time to make the leap from dictionaries and tuples to a hierarchy of classes.

### Helper classes

In [ ]:
Grade = namedtuple("Grade", ("score", "weight"))

* Write a class to represent a single subject that contains a set of grades

In [ ]:
class Subject:
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

* Then you would write a class to represent a set of subjects that are being studies by a single student.

In [ ]:
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
        
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count   

* Fianlly, you'd write a container for all of the students keyed dynamically by their names.

In [ ]:
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)
        
    def get_student(self, name):
        return self._students[name]

In [ ]:
book = Gradebook()
H = book.get_student("student_H")

math = H.get_subject("Math")
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = H.get_subject("Gym")
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

In [ ]:
print(H.average_grade())

### Things to Remember

* Avoid making dictionaries with values that are other dictionaries or long tuples.
* Use `namedtuple` for lightweight, immutable data containers before you need the flexibility of a full class.
* Move your bookkeeping code to use multiple helper classes when your internal state dictionaries get complicated.